# SurrealDB-ORM -- Setup & Prerequisites

This notebook walks you through setting up your environment to use the SurrealDB-ORM.
All other example notebooks assume this setup has been completed.

## 1. Start SurrealDB with Docker

The ORM connects to a running SurrealDB instance. The project ships with a Docker Compose
file that starts SurrealDB for you.

Open a terminal **in the project root** and run:

```bash
# Development instance (port 8000, persistent data)
docker compose -f devops/docker-compose.yml up -d surrealdb

# Or for tests (port 8001, ephemeral data):
docker compose -f devops/docker-compose.yml up -d surrealdb-test
```

You can also use `make db-dev` or `make db-up` as shortcuts.

> **Tip:** Run `docker ps` to confirm the container is running before continuing.

## 2. Environment Variables

The project uses a `.env` file for connection settings. If you haven't already,
copy the example file:

```bash
cp .env.example .env
```

The defaults work out of the box with the Docker Compose setup:

| Variable             | Default                    |
|----------------------|----------------------------|
| `SURREALDB_URL`      | `ws://localhost:8000`      |
| `SURREALDB_USER`     | `root`                     |
| `SURREALDB_PASS`     | `root`                     |
| `SURREALDB_NAMESPACE`| `ns`                       |
| `SURREALDB_DATABASE` | `db`                       |

> **Note:** The ORM uses HTTP under the hood. If your `.env` specifies a `ws://` URL,
> it is automatically converted to `http://` for HTTP connections.

## 3. Configure the ORM Connection

The cell below loads environment variables and registers the default connection
with `SurrealDBConnectionManager`. Every notebook in this series starts with a
similar cell.

In [3]:
import os
import sys

# Add the project root to the Python path so we can import from `src`
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

# Load .env file if present
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

# Read connection parameters from environment (with sensible defaults)
SURREALDB_URL = os.getenv("SURREALDB_URL", "ws://localhost:8000")
SURREALDB_USER = os.getenv("SURREALDB_USER", "root")
SURREALDB_PASS = os.getenv("SURREALDB_PASS", "root")
SURREALDB_NAMESPACE = os.getenv("SURREALDB_NAMESPACE", "ns")
SURREALDB_DATABASE = os.getenv("SURREALDB_DATABASE", "db")

# Register the default connection
SurrealDBConnectionManager.set_connection(
    SURREALDB_URL,
    SURREALDB_USER,
    SURREALDB_PASS,
    SURREALDB_NAMESPACE,
    SURREALDB_DATABASE,
)

print(f"Connection configured: {SURREALDB_URL}")
print(f"Namespace: {SURREALDB_NAMESPACE} | Database: {SURREALDB_DATABASE}")

Connection configured: ws://localhost:8000
Namespace: ns | Database: db


## 4. Verify the Connection

Let's run a quick query to make sure we can reach the database.

In [4]:
# Obtain the HTTP client and run a diagnostic query
client = await SurrealDBConnectionManager.get_client()
result = await client.query("INFO FOR DB;")

print("Connected successfully!")
print(f"Database info: {result}")

Connected successfully!
Database info: QueryResponse(results=[QueryResult(status=<ResponseStatus.OK: 'OK'>, result={'accesses': {}, 'analyzers': {}, 'apis': {}, 'configs': {}, 'functions': {}, 'models': {}, 'params': {}, 'tables': {}, 'users': {}}, time='424.728µs')], raw=[{'result': {'accesses': {}, 'analyzers': {}, 'apis': {}, 'configs': {}, 'functions': {}, 'models': {}, 'params': {}, 'tables': {}, 'users': {}}, 'status': 'OK', 'time': '424.728µs'}])


## You're Ready!

If you see the database info above, everything is working. Each notebook in
this series starts with a similar setup cell so you can run them independently.

Next notebooks:

| Notebook                     | Topic                            |
|------------------------------|----------------------------------|
| `01_models_crud.ipynb`       | Models & CRUD Operations         |
| `02_queries_filters.ipynb`   | Queries & Filters                |
| `03_aggregations.ipynb`      | Aggregations & GROUP BY          |
| `04_relations_graph.ipynb`   | Relations & Graph Traversal      |